In [5]:
# =============================================================================
# Training and plotting of final ANN
# =============================================================================

import pandas as pd;
import numpy as np;
import sklearn
from sklearn.model_selection import train_test_split, cross_validate, cross_val_predict;
from sklearn.preprocessing import MinMaxScaler;
from sklearn import metrics;
from sklearn.model_selection import TimeSeriesSplit;

# empty list to append metric values
mae_cv = []
mse_cv = []
mae_gen = []
mse_gen  =[]
rmse_gen = []
mae_nor = []
mae_spi = []
mse_nor = []
mse_spi = []
rmse_nor = []
rmse_spi = []

# import data
data = pd.read_csv('Data_set_1_smaller.csv', index_col = 0)

# 3 months
data = data.loc[data.index > 2018090000, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# Divide features and labels
X = data.iloc[:, 0:15]
y = data.loc[:, 'Offers']

X.fillna(method = 'ffill', inplace = True)
y.fillna(method = 'ffill', inplace = True)

X = X.astype('float64')
X = X.round(20)

# divide data into train and test with 20% test data
X_train, X_test, y_train, y_test = train_test_split(
         X, y, test_size = 0.15, shuffle=False)

# feature scaling
sc_X = MinMaxScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

import keras
from keras.models import Sequential # to initialise the NN
from keras.layers import Dense # to create layers
from keras.layers import Dropout
from keras import initializers
from keras import optimizers
from keras.wrappers.scikit_learn import KerasRegressor

# possible debug
X_train = np.nan_to_num(X_train)
X_test = np.nan_to_num(X_test)

def regressor_tunning(n_hidden = 5, 
                      n_neurons = 40, 
                      kernel_initializer = "he_normal",
                      bias_initializer = initializers.Ones()):
    model = Sequential()
    model.add(Dense(units = n_neurons, input_dim = 15))
    model.add(keras.layers.LeakyReLU(alpha = 0.2))
    model.add(Dropout(rate = 0.3))
    for layer in range(n_hidden):
        model.add(Dense(units = n_neurons))
        model.add(keras.layers.LeakyReLU(alpha = 0.2))
        model.add(Dropout(rate = 0.3))
    model.add(Dense(units = 1, activation = 'linear'))
    optimizer = optimizers.Adamax(lr = 0.001)
    model.compile(loss = 'mse', optimizer = optimizer, metrics = ['mse', 'mae'])
    return model

tscv = TimeSeriesSplit(n_splits = 10)

hist_list = pd.DataFrame()
count = 1
    
regressor = regressor_tunning()
    
for train_index, test_index in tscv.split(X_train):
    X_train_split, X_test_split = X_train[train_index], X_train[test_index]
    y_train_split, y_test_split = y_train[train_index], y_train[test_index]
    hist = regressor.fit(X_train_split, y_train_split, batch_size = 10, epochs = 100)
    hist_list = hist_list.append(hist.history, ignore_index = True)
    print(count)
    count = count + 1

a = []
b = []

for i in range(len(hist_list.mse)):
    a.append(np.mean(hist_list.mse[i]))
    b.append(np.mean(hist_list.mae[i]))

mse_cv.append(np.mean(a))
mae_cv.append(np.mean(b))

# predict for X_test  
y_pred = regressor.predict(X_test)

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae

rmse_error = mse(y_test, y_pred, squared = False)
mse_error = mse(y_test, y_pred) # 1479.61335
mae_error = mae(y_test, y_pred) # 23.1525

rmse_gen.append(rmse_error)
mse_gen.append(mse_error)
mae_gen.append(mae_error)

# =============================================================================
# Metrics evaluation on spike regions
# =============================================================================

y_spike_occ = pd.read_csv('Spike_binary_1std.csv', usecols = [6])

# create array same size as y_test
y_spike_occ = y_spike_occ.iloc[- len(y_test):]
y_spike_occ = pd.Series(y_spike_occ.iloc[:,0]).values


# smal adjustment
y_test.replace(0, 0.0001,inplace = True)


# select y_pred and y_test only for regions with spikes
y_test_spike = (y_test.T * y_spike_occ).T
y_pred_spike = (y_pred.T * y_spike_occ).T
y_test_spike = y_test_spike[y_test_spike != 0]
y_pred_spike = y_pred_spike[y_pred_spike != 0]

# calculate metric
rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
mse_spike = mse(y_test_spike, y_pred_spike)
mae_spike = mae(y_test_spike, y_pred_spike)

rmse_spi.append(rmse_spike)
mse_spi.append(mse_spike)
mae_spi.append(mae_spike)

# =============================================================================
# Metric evaluation on normal regions
# =============================================================================

# inverse y_spike_occ so the only normal occurences are chosen
y_normal_occ = (y_spike_occ - 1) * (-1)

# sanity check
y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

# select y_pred and y_test only for normal regions
y_test_normal = (y_test.T * y_normal_occ).T
y_pred_normal = (y_pred.T * y_normal_occ).T
y_test_normal = y_test_normal[y_test_normal != 0.00]
y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

# calculate metric
rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
mse_normal = mse(y_test_normal, y_pred_normal)
mae_normal = mae(y_test_normal, y_pred_normal)

rmse_nor.append(rmse_normal)
mse_nor.append(mse_normal)
mae_nor.append(mae_normal)
# Save


results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor})



C:\Users\maria\anaconda3\lib\site-packages\pandas\core\frame.py:4153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


Epoch 1/100
46/46 [==============================] - 0s 1ms/step - loss: 13371.1787 - mse: 13371.1787 - mae: 109.7077
Epoch 2/100
46/46 [==============================] - 0s 2ms/step - loss: 12735.8242 - mse: 12735.8242 - mae: 106.7546
Epoch 3/100
46/46 [==============================] - 0s 1ms/step - loss: 9443.0430 - mse: 9443.0430 - mae: 88.6662
Epoch 4/100
46/46 [==============================] - 0s 1ms/step - loss: 3344.0889 - mse: 3344.0889 - mae: 43.5235
Epoch 5/100
46/46 [==============================] - 0s 2ms/step - loss: 2836.7095 - mse: 2836.7095 - mae: 38.9562
Epoch 6/100
46/46 [==============================] - 0s 1ms/step - loss: 2584.3254 - mse: 2584.3257 - mae: 37.1237
Epoch 7/100
46/46 [==============================] - 0s 1ms/step - loss: 2949.6677 - mse: 2949.6677 - mae: 39.5813
Epoch 8/100
46/46 [==============================] - 0s 1ms/step - loss: 2950.9292 - mse: 2950.9292 - mae: 38.7103
Epoch 9/100
46/46 [==============================] - 0s 2ms/step - loss: 2

46/46 [==============================] - 0s 1ms/step - loss: 1943.8752 - mse: 1943.8752 - mae: 30.3526
Epoch 72/100
46/46 [==============================] - 0s 2ms/step - loss: 2048.3870 - mse: 2048.3872 - mae: 31.0438
Epoch 73/100
46/46 [==============================] - 0s 2ms/step - loss: 2095.9602 - mse: 2095.9602 - mae: 31.4540
Epoch 74/100
46/46 [==============================] - 0s 2ms/step - loss: 1912.2261 - mse: 1912.2261 - mae: 30.8313
Epoch 75/100
46/46 [==============================] - 0s 2ms/step - loss: 2208.3594 - mse: 2208.3594 - mae: 32.0626
Epoch 76/100
46/46 [==============================] - 0s 2ms/step - loss: 2076.9805 - mse: 2076.9805 - mae: 31.2857
Epoch 77/100
46/46 [==============================] - 0s 2ms/step - loss: 2143.4146 - mse: 2143.4146 - mae: 31.6056
Epoch 78/100
46/46 [==============================] - 0s 2ms/step - loss: 1983.1910 - mse: 1983.1910 - mae: 30.4066
Epoch 79/100
46/46 [==============================] - 0s 2ms/step - loss: 2091.6201 -

92/92 [==============================] - 0s 2ms/step - loss: 4171.0557 - mse: 4171.0557 - mae: 32.2798
Epoch 41/100
92/92 [==============================] - 0s 2ms/step - loss: 4205.0171 - mse: 4205.0166 - mae: 31.6480
Epoch 42/100
92/92 [==============================] - 0s 2ms/step - loss: 4041.6711 - mse: 4041.6711 - mae: 31.7852
Epoch 43/100
92/92 [==============================] - 0s 2ms/step - loss: 4038.7410 - mse: 4038.7410 - mae: 31.8881
Epoch 44/100
92/92 [==============================] - 0s 2ms/step - loss: 4126.8257 - mse: 4126.8257 - mae: 32.7905
Epoch 45/100
92/92 [==============================] - 0s 2ms/step - loss: 4070.1812 - mse: 4070.1812 - mae: 32.4629
Epoch 46/100
92/92 [==============================] - 0s 2ms/step - loss: 4068.9656 - mse: 4068.9656 - mae: 31.6370
Epoch 47/100
92/92 [==============================] - 0s 2ms/step - loss: 4161.5151 - mse: 4161.5151 - mae: 32.2417
Epoch 48/100
92/92 [==============================] - 0s 2ms/step - loss: 3944.1199 -

137/137 [==============================] - 0s 2ms/step - loss: 3505.2769 - mse: 3505.2769 - mae: 32.0497
Epoch 11/100
137/137 [==============================] - 0s 2ms/step - loss: 3401.7371 - mse: 3401.7371 - mae: 31.4253
Epoch 12/100
137/137 [==============================] - 0s 2ms/step - loss: 3600.2166 - mse: 3600.2166 - mae: 33.0301
Epoch 13/100
137/137 [==============================] - 0s 2ms/step - loss: 3438.1567 - mse: 3438.1567 - mae: 32.2156
Epoch 14/100
137/137 [==============================] - 0s 2ms/step - loss: 3385.4075 - mse: 3385.4075 - mae: 32.4423
Epoch 15/100
137/137 [==============================] - 0s 2ms/step - loss: 3315.4812 - mse: 3315.4812 - mae: 31.5735
Epoch 16/100
137/137 [==============================] - 0s 2ms/step - loss: 3434.6541 - mse: 3434.6541 - mae: 32.0561
Epoch 17/100
137/137 [==============================] - 0s 2ms/step - loss: 3521.6396 - mse: 3521.6396 - mae: 31.9775
Epoch 18/100
137/137 [==============================] - 0s 2ms/step -

...

318/318 [==============================] - 1s 2ms/step - loss: 2071.6558 - mse: 2071.6558 - mae: 27.1119
Epoch 94/100
318/318 [==============================] - 0s 1ms/step - loss: 2034.1005 - mse: 2034.1005 - mae: 27.0972
Epoch 95/100
318/318 [==============================] - 0s 1ms/step - loss: 2029.5206 - mse: 2029.5206 - mae: 27.0554
Epoch 96/100
318/318 [==============================] - 0s 1ms/step - loss: 2111.7151 - mse: 2111.7151 - mae: 27.3864
Epoch 97/100
318/318 [==============================] - 1s 2ms/step - loss: 2000.0139 - mse: 2000.0139 - mae: 26.9221
Epoch 98/100
318/318 [==============================] - 1s 2ms/step - loss: 2122.3489 - mse: 2122.3489 - mae: 27.3024
Epoch 99/100
318/318 [==============================] - 0s 2ms/step - loss: 2081.4124 - mse: 2081.4124 - mae: 27.4411
Epoch 100/100
318/318 [==============================] - 1s 2ms/step - loss: 2050.0361 - mse: 2050.0364 - mae: 26.6847
7
Epoch 1/100
363/363 [==============================] - 1s 2ms/step

363/363 [==============================] - 1s 2ms/step - loss: 2502.9568 - mse: 2502.9563 - mae: 26.9269
Epoch 62/100
363/363 [==============================] - 1s 2ms/step - loss: 2479.0271 - mse: 2479.0271 - mae: 26.5900
Epoch 63/100
363/363 [==============================] - 1s 2ms/step - loss: 2487.8882 - mse: 2487.8882 - mae: 26.9287
Epoch 64/100
363/363 [==============================] - 1s 2ms/step - loss: 2489.0110 - mse: 2489.0110 - mae: 27.0249
Epoch 65/100
363/363 [==============================] - 1s 1ms/step - loss: 2463.1670 - mse: 2463.1670 - mae: 26.9884
Epoch 66/100
363/363 [==============================] - 1s 2ms/step - loss: 2472.6074 - mse: 2472.6074 - mae: 26.8339
Epoch 67/100
363/363 [==============================] - 1s 2ms/step - loss: 2547.1335 - mse: 2547.1335 - mae: 27.1244
Epoch 68/100
363/363 [==============================] - 1s 2ms/step - loss: 2472.5051 - mse: 2472.5051 - mae: 26.7809
Epoch 69/100
363/363 [==============================] - 1s 2ms/step -

408/408 [==============================] - 1s 2ms/step - loss: 2597.5881 - mse: 2597.5881 - mae: 27.8329
Epoch 30/100
408/408 [==============================] - 1s 2ms/step - loss: 2591.8337 - mse: 2591.8337 - mae: 27.8178
Epoch 31/100
408/408 [==============================] - 1s 1ms/step - loss: 2576.7678 - mse: 2576.7678 - mae: 27.6638
Epoch 32/100
408/408 [==============================] - 1s 2ms/step - loss: 2614.9297 - mse: 2614.9297 - mae: 27.6001
Epoch 33/100
408/408 [==============================] - 1s 2ms/step - loss: 2587.2344 - mse: 2587.2344 - mae: 27.6526
Epoch 34/100
408/408 [==============================] - 1s 2ms/step - loss: 2589.2786 - mse: 2589.2786 - mae: 27.6221
Epoch 35/100
408/408 [==============================] - 1s 2ms/step - loss: 2623.3721 - mse: 2623.3721 - mae: 27.8341
Epoch 36/100
408/408 [==============================] - 1s 2ms/step - loss: 2573.6826 - mse: 2573.6826 - mae: 27.4441
Epoch 37/100
408/408 [==============================] - 1s 2ms/step -

408/408 [==============================] - 1s 2ms/step - loss: 2489.2036 - mse: 2489.2039 - mae: 27.3804
Epoch 98/100
408/408 [==============================] - 1s 2ms/step - loss: 2553.8022 - mse: 2553.8022 - mae: 27.2370
Epoch 99/100
408/408 [==============================] - 1s 2ms/step - loss: 2546.3691 - mse: 2546.3691 - mae: 27.1770
Epoch 100/100
408/408 [==============================] - 1s 2ms/step - loss: 2537.8799 - mse: 2537.8799 - mae: 27.6731
9
Epoch 1/100
453/453 [==============================] - 1s 2ms/step - loss: 2353.8298 - mse: 2353.8298 - mae: 26.6713
Epoch 2/100
453/453 [==============================] - 1s 2ms/step - loss: 2408.0298 - mse: 2408.0298 - mae: 26.5696: 0s - loss: 2993.3552 - mse: 29
Epoch 3/100
453/453 [==============================] - 1s 2ms/step - loss: 2416.3540 - mse: 2416.3540 - mae: 26.8669
Epoch 4/100
453/453 [==============================] - 1s 2ms/step - loss: 2395.1926 - mse: 2395.1926 - mae: 26.5513
Epoch 5/100
453/453 [=================

453/453 [==============================] - 1s 2ms/step - loss: 2351.7942 - mse: 2351.7942 - mae: 26.6746
Epoch 66/100
453/453 [==============================] - 1s 2ms/step - loss: 2399.4358 - mse: 2399.4358 - mae: 26.6177
Epoch 67/100
453/453 [==============================] - 1s 2ms/step - loss: 2293.6184 - mse: 2293.6184 - mae: 26.1894
Epoch 68/100
453/453 [==============================] - 1s 2ms/step - loss: 2326.7434 - mse: 2326.7434 - mae: 26.2625
Epoch 69/100
453/453 [==============================] - 1s 2ms/step - loss: 2388.0288 - mse: 2388.0288 - mae: 26.6601
Epoch 70/100
453/453 [==============================] - 1s 2ms/step - loss: 2325.6541 - mse: 2325.6541 - mae: 26.3032
Epoch 71/100
453/453 [==============================] - 1s 2ms/step - loss: 2350.8540 - mse: 2350.8540 - mae: 26.6328
Epoch 72/100
453/453 [==============================] - 1s 2ms/step - loss: 2340.1455 - mse: 2340.1455 - mae: 26.5243
Epoch 73/100
453/453 [==============================] - 1s 2ms/step -

In [6]:
results

,rmse_general,mae_general,rmse_spike,mae_spike,rmse_normal,mae_normal
0,30.939864,21.999466,66.545431,55.02067,20.851979,17.028997


In [7]:
results.to_csv('Results_100epochs_10batch_10splits_0.3_dropout')